## Part 1: Preprocessing

In [224]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [225]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [226]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [227]:
# Create a list of at least 10 column names to use as X data
X = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 
     'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[X]

# Show the data types for X_df
X_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [228]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)


In [229]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


In [230]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [231]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoded_department = enc.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_dept = enc.transform(y_train[['Department']])
y_test_encoded_dept = enc.transform(y_test[['Department']])
y_train_encoded_dept



array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [232]:
# Create a OneHotEncoder for the Attrition column
# enc = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoded_attrition = enc.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_att = enc.transform(y_train[['Attrition']])
y_test_encoded_att = enc.transform(y_test[['Attrition']])
y_train_encoded_att



array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

## Create, Compile, and Train the Model

In [233]:
# Find the number of columns in the X training data
X_train.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(X_train.shape[1],), name='input')

# Create at least two shared layers
shared_layer1 = layers.Dense(units=64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(units=128, activation='relu')(shared_layer1)


In [234]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(units=32, activation='relu')(shared_layer2)

# Create the output layer with name 'department_output'
department_output_layer = layers.Dense(units=3, activation='softmax', name='department')(department_hidden_layer)

In [235]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(units=32, activation='relu')(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(units=2, activation='softmax', name='attrition')(attrition_hidden_layer)

In [236]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])


# Compile the model
model.compile(optimizer='adam',
              loss={'department': 'categorical_crossentropy', 'attrition': 'categorical_crossentropy'},
              metrics={'department': 'accuracy', 'attrition': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 11)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_57 (Dense)    │ (None, 64)        │        768 │ input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_58 (Dense)    │ (None, 128)       │      8,320 │ dense_57[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_59 (Dense)    │ (None, 32)        │      4,128 │ dense_58[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_60 (Dense)    │ (None, 32)        │      4,128 │ dense_58[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department (Dense)  │ (None, 3)         │         99 │ dense_59[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition (Dense)   │ (None, 2)         │         66 │ dense_60[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,509 (68.39 KB)

 Trainable params: 17,509 (68.39 KB)

 Non-trainable params: 0 (0.00 B)

In [237]:
print(X_train.shape)
print(y_train_encoded_dept.shape)
print(y_train_encoded_att.shape)

(1102, 11)
(1102, 3)
(1102, 2)


In [238]:
# Train the model
model.fit(X_train_scaled,
            {'department': y_train_encoded_dept, 'attrition': y_train_encoded_att},
            epochs=100,
            shuffle=True,
            verbose=2)




Epoch 1/100
35/35 - 1s - 16ms/step - attrition_accuracy: 0.8076 - attrition_loss: 0.4774 - department_accuracy: 0.6434 - department_loss: 0.8268 - loss: 1.3130
Epoch 2/100
35/35 - 0s - 964us/step - attrition_accuracy: 0.8276 - attrition_loss: 0.4168 - department_accuracy: 0.6633 - department_loss: 0.7578 - loss: 1.1763
Epoch 3/100
35/35 - 0s - 954us/step - attrition_accuracy: 0.8330 - attrition_loss: 0.3949 - department_accuracy: 0.6652 - department_loss: 0.7420 - loss: 1.1361
Epoch 4/100
35/35 - 0s - 935us/step - attrition_accuracy: 0.8475 - attrition_loss: 0.3873 - department_accuracy: 0.6670 - department_loss: 0.7310 - loss: 1.1142
Epoch 5/100
35/35 - 0s - 1ms/step - attrition_accuracy: 0.8593 - attrition_loss: 0.3668 - department_accuracy: 0.6733 - department_loss: 0.7283 - loss: 1.0925
Epoch 6/100
35/35 - 0s - 962us/step - attrition_accuracy: 0.8621 - attrition_loss: 0.3553 - department_accuracy: 0.6788 - department_loss: 0.7243 - loss: 1.0803
Epoch 7/100
35/35 - 0s - 990us/step -

In [239]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department': y_test_encoded_dept, 'attrition': y_test_encoded_att})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_accuracy: 0.8104 - attrition_loss: 1.4280 - department_accuracy: 0.5082 - department_loss: 3.0466 - loss: 4.4855  


[4.501087188720703,
 3.0222761631011963,
 1.4078632593154907,
 0.8152173757553101,
 0.51902174949646]

In [240]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[4]}")
print(f"Attrition Accuracy: {test_results[3]}")

Department Accuracy: 0.51902174949646
Attrition Accuracy: 0.8152173757553101


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is not the best metric with an imbalanced dataset.
2. I chose softmax which is suitable for multi-class classification problems as it converts the raw output scores into probabilities that sum to 1, allowing for the interpretation of the output as the likelihood of each class.
3. A couple ways this model could be improved would be through hyperparameter tuning and data augmentation. 